In [67]:
import cv2 as cv
import paho.mqtt.client as mqtt

In [16]:
from PIL import Image
import sys
import os
import urllib
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
import numpy as np
import time
#from tf_trt_models.detection import download_detection_model, build_detection_graph

%matplotlib inline

### Load the pre-trained SSD model

In [17]:
# https://github.com/yeephycho/tensorflow-face-detection
FROZEN_GRAPH_NAME = 'data/frozen_inference_graph_face.pb'
!wget https://github.com/yeephycho/tensorflow-face-detection/blob/master/model/frozen_inference_graph_face.pb?raw=true -O {FROZEN_GRAPH_NAME}

--2020-06-03 13:30:38--  https://github.com/yeephycho/tensorflow-face-detection/blob/master/model/frozen_inference_graph_face.pb?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/yeephycho/tensorflow-face-detection/raw/master/model/frozen_inference_graph_face.pb [following]
--2020-06-03 13:30:38--  https://github.com/yeephycho/tensorflow-face-detection/raw/master/model/frozen_inference_graph_face.pb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yeephycho/tensorflow-face-detection/master/model/frozen_inference_graph_face.pb [following]
--2020-06-03 13:30:38--  https://raw.githubusercontent.com/yeephycho/tensorflow-face-detection/master/model/frozen_inference_graph_face.pb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)

### Load the frozen graph

In [18]:
output_dir=''
frozen_graph = tf.compat.v1.GraphDef()
with open(os.path.join(output_dir, FROZEN_GRAPH_NAME), 'rb') as f:
  frozen_graph.ParseFromString(f.read())

### A few magical constants...

In [19]:
# https://github.com/NVIDIA-AI-IOT/tf_trt_models/blob/master/tf_trt_models/detection.py
INPUT_NAME='image_tensor'
BOXES_NAME='detection_boxes'
CLASSES_NAME='detection_classes'
SCORES_NAME='detection_scores'
MASKS_NAME='detection_masks'
NUM_DETECTIONS_NAME='num_detections'

input_names = [INPUT_NAME]
output_names = [BOXES_NAME, CLASSES_NAME, SCORES_NAME, NUM_DETECTIONS_NAME]

### Create a session and load the graph

In [20]:
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.compat.v1.Session(config=tf_config)

# use this if you want to try on the optimized TensorRT graph
# Note that this will take a while
# tf.import_graph_def(trt_graph, name='')

# use this if you want to try directly on the frozen TF graph
# this is much faster
tf.import_graph_def(frozen_graph, name='')

tf_input = tf_sess.graph.get_tensor_by_name(input_names[0] + ':0')
tf_scores = tf_sess.graph.get_tensor_by_name('detection_scores:0')
tf_boxes = tf_sess.graph.get_tensor_by_name('detection_boxes:0')
tf_classes = tf_sess.graph.get_tensor_by_name('detection_classes:0')
tf_num_detections = tf_sess.graph.get_tensor_by_name('num_detections:0')

In [10]:
# suppress boxes that are below the threshold.. 
DETECTION_THRESHOLD = 0.5

### Function to capture faces: preprocess the image, run network on the image, crop the image to contain only the face, then send the face to message broker

In [88]:
def capture_faces(image):
    
    image_resized = cv.resize(image, (300,300), interpolation = cv.INTER_AREA)
    image = np.array(image)
    
    scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={
        tf_input: image_resized[None, ...]
    })

    boxes = boxes[0] # index by 0 to remove batch dimension
    scores = scores[0]
    classes = classes[0]
    num_detections = num_detections[0]
    
    #fig = plt.figure()
    #ax = fig.add_subplot(1, 1, 1)

    # plot boxes exceeding score threshold
    for i in range(int(num_detections)):
        if scores[i] < DETECTION_THRESHOLD:
            continue
        # scale box to image coordinates
        box = boxes[i] * np.array([image.shape[0], image.shape[1], image.shape[0], image.shape[1]])

        # display rectangle
        #patch = patches.Rectangle((box[1], box[0]), box[3] - box[1], box[2] - box[0], color='g', alpha=0.3)
        #ax.add_patch(patch)

        # display class index and score
        #plt.text(x=box[1] + 10, y=box[2] - 10, s='%d (%0.2f) ' % (classes[i], scores[i]), color='w')
        
        # capture the face
        face = image[int(box[0]):int(box[2]),int(box[1]):int(box[3])]
        #ax.imshow(face)
        
        rc, jpg=cv.imencode('.png', face)
        msg = jpg.tobytes()
        
        client.publish("faces_detected", payload=msg)


### Capture faces for a fixed number of times to calculate the FPS

In [92]:
# create a new client instance and connect to the local broker.
client = mqtt.Client()
client.connect("172.18.0.2")

cap = cv.VideoCapture(1)    

t0 = time.time()
for i in range(50):
    ret, frame = cap.read()
    frame_fix = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    capture_faces(frame_fix)
t1 = time.time()

print("FPS: ", int(50/(t1-t0)))

cap.release()

FPS:  8


### Captue faces for cloud storage

In [ ]:
# create a new client instance and connect to the local broker.
client = mqtt.Client()
client.connect("172.18.0.2")

cap = cv.VideoCapture(1)    

while(True):
    ret, frame = cap.read()
    frame_fix = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    capture_faces(frame_fix)
    cv.waitKey(1)

cap.release()

### Close session to release resources

In [14]:
tf_sess.close()